# Импорт

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split


from Classes.CartTreeClassifier import CartTreeClassifier
from Classes.CartTreeTeacher import CartTreeTeacher

from Functions.metrics import *
from Functions.split_functions import *
from Functions.spreading_functions import *
from Functions.search_split_functions import *

from Functions.metrics import accuracy

from Functions.errorFunctions import MAE, MAPE, MSE, RMSE, R2


# Загрузка датасета для регрессии.

In [2]:
models = pd.read_csv('../Data/DataLaba1/energy_task_moded.csv')


# Разбиваем на выборку

In [3]:
X_train, X_test, y_train, y_test = train_test_split(models.drop('Appliances', axis=1), models['Appliances'], test_size=0.5)


# Cart Tree Regressor

## On Small Data

In [4]:
ctc = CartTreeClassifier()
ctt = CartTreeTeacher(ctc, lambda a, b: classic_split(a, b[1], b[2]), search_params_that_equiv_to_majority_class, lambda a, b: a**2 and b>0, 10)
ctt.teach(X_train.to_numpy()[0:1000], y_train.to_numpy()[0:1000])

In [5]:
y_predicted = ctc.predict(X_test.to_numpy()[0:1000])

In [6]:
my_line_regression_classic_errors = {'MAE': MAE(y_predicted, y_test[0:1000]),
                                     'RMSE': RMSE(y_predicted, y_test[0:1000]),
                                     'MSE': MSE(y_predicted, y_test[0:1000]),
                                     'MAPE': MAPE(y_predicted, y_test[0:1000]),
                                     'R^2': R2(y_predicted, np.array(y_test[0:1000]))
                                     }

my_line_regression_classic_errors

{'MAE': 49.69,
 'RMSE': 109.96954123756268,
 'MSE': 12093.3,
 'MAPE': 0.41351485125829546,
 'R^2': 0.9988229923256293}